In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Kidney_Papillary_Cell_Carcinoma/GSE68606'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"caArray_dobbi-00100: Interlaboratory comparability study of cancer gene expression analysis using oligonucleotide microarrays"
!Series_summary	"A key step in bringing gene expression data into clinical practice is the conduct of large studies to confirm preliminary models. The performance of such confirmatory studies and the transition to clinical practice requires that microarray data from different laboratories are comparable and reproducible. We designed a study to assess the comparability of data from four laboratories that will conduct a larger microarray profiling confirmation project in lung adenocarcinomas. To test the feasibility of combining data across laboratories, frozen tumor tissues, cell line pellets, and purified RNA samples were analyzed at each of the four laboratories. Samples of each type and several subsamples from each tumor and each cell line were blinded before being distributed. The laboratories followed a common protocol

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Checking if dataset contains gene expression data
# Based on the background information, this dataset uses Affymetrix Human Genome U133A arrays which provide gene expression data.
is_gene_available = True

# Determining the keys for the variables of interest
# 'Kidney_Papillary_Cell_Carcinoma' can be inferred from 'disease state'
trait_row = 1  # inferred from 'disease state'
# 'age' is directly available
age_row = 6  # explicitly available in sample characteristics dictionary
# 'gender' is directly available 
gender_row = 5  # explicitly available in sample characteristics dictionary

# Functions to convert variables to the correct data type
def convert_trait(value):
    extracted_value = value.split(':')[-1].strip().lower()
    if "papillary" in extracted_value:
        return 1
    if extracted_value == '--':
        return None
    return 0

def convert_age(value):
    extracted_value = value.split(':')[-1].strip()
    if extracted_value == '--':
        return None
    return int(extracted_value)

def convert_gender(value):
    extracted_value = value.split(':')[-1].strip().lower()
    if extracted_value == 'female':
        return 0
    elif extracted_value == 'male':
        return 1
    else:
        return None

# Save cohort information
save_cohort_info('GSE68606', './preprocessed/Kidney_Papillary_Cell_Carcinoma/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical feature extraction and saving
selected_clinical_data = geo_select_clinical_features(clinical_data, 'Kidney_Papillary_Cell_Carcinoma', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
csv_path = './preprocessed/Kidney_Papillary_Cell_Carcinoma/trait_data/GSE68606.csv'
selected_clinical_data.to_csv(csv_path)
print(preview_df(selected_clinical_data))


{'GSM1676864': [None, None, None], 'GSM1676865': [None, None, None], 'GSM1676866': [0, None, 0], 'GSM1676867': [None, None, None], 'GSM1676868': [None, None, None], 'GSM1676869': [None, None, None], 'GSM1676870': [None, None, None], 'GSM1676871': [None, None, None], 'GSM1676872': [None, None, None], 'GSM1676873': [None, None, None], 'GSM1676874': [0, 67, 1], 'GSM1676875': [0, 66, 1], 'GSM1676876': [0, 72, 1], 'GSM1676877': [0, 56, 0], 'GSM1676878': [0, 48, 0], 'GSM1676879': [None, None, None], 'GSM1676880': [None, None, None], 'GSM1676881': [0, None, None], 'GSM1676882': [0, None, None], 'GSM1676883': [0, None, None], 'GSM1676884': [0, None, None], 'GSM1676885': [0, None, None], 'GSM1676886': [0, None, None], 'GSM1676887': [0, None, None], 'GSM1676888': [None, None, None], 'GSM1676889': [0, None, None], 'GSM1676890': [0, None, None], 'GSM1676891': [0, None, None], 'GSM1676892': [0, None, None], 'GSM1676893': [0, None, None], 'GSM1676894': [0, None, None], 'GSM1676895': [0, None, None],

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1598_g_at', '160020_at', '1729_at', '1773_at', '177_at',
       '179_at', '1861_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Deciding the keys for identifiers and gene symbols.
identifier_key = 'ID'
gene_symbol_key = 'Gene Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes using the 'get_gene_mapping' function from the library.
gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping with the 'apply_gene_mapping' function from the library, and name the resulting gene expression dataframe "gene_data".
gene_data = apply_gene_mapping(gene_data, gene_mapping)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Kidney_Papillary_Cell_Carcinoma/gene_data/GSE68606.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Kidney_Papillary_Cell_Carcinoma')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE68606', './preprocessed/Kidney_Papillary_Cell_Carcinoma/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Kidney_Papillary_Cell_Carcinoma/GSE68606.csv'
    unbiased_merged_data.to_csv(csv_path)


Quartiles for 'Kidney_Papillary_Cell_Carcinoma':
  25%: 0.0
  50% (Median): 0.0
  75%: 0.0
Min: 0.0
Max: 0.0
The distribution of the feature 'Kidney_Papillary_Cell_Carcinoma' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 56.0
  50% (Median): 66.0
  75%: 67.0
Min: 48.0
Max: 72.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 8 occurrences. This represents 40.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

